## Simulation 70k

This file will generate simulations of different types of architecture for the $70$ k dataset. The various architectures are derived in handwritten notes. The different phenotypes that are simulated will be tested on $4$ different models:

**XGBoost model**

**Ridge regression**

**Ridge regression with PC's as input**

**Bayesian animal model with INLA**

The architecture has form:

$$p(u_{j}|\sigma_{G}^{2}) = 0 \cdot \pi_{0} + \sum_{i=1}^{3}\pi_{i} \cdot N(0, \sigma_{ui}^{2}\sigma_{G}^{2})$$

Naming convention for the phenotype file will be: **SimPhenoArch_i**, where $i$ denotes the type of architecture used from the notes.


In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow.feather as feather

Load SNP data and do some data manipulation.

In [2]:
SNP_matrix = pd.read_feather("C:/Users/gard_/Documents/MasterThesis/Code/Data/SNP_70k.feather")

# Shows duplicated ringnr's:
print(SNP_matrix["ringnr"].duplicated().any())

print("shape: ", SNP_matrix.shape, "\n")

# Remove duplicated ringnr's by keeping first instance:
SNP_matrix = SNP_matrix.drop_duplicates(subset="ringnr", keep="first")

# Reset the index to remove any gaps (i.e., ensure sequential index)
SNP_matrix = SNP_matrix.reset_index(drop=True)

print("shape: ", SNP_matrix.shape, "\n")

print(SNP_matrix["ringnr"].duplicated().any())

# M_tilde contains only the SNPs
M_tilde = SNP_matrix.drop(columns=["ringnr"])

# Replace NA values with 0
M_tilde.fillna(0, inplace=True)

#Mean-center M
M_tilde = M_tilde - M_tilde.mean(axis=0)

True
shape:  (12059, 65239) 

shape:  (11997, 65239) 

False


Function for generating simulated effect sizes.

In [ ]:
def get_u(p, arch):
    """
    Generate effect sizes u. 'p' is number of columns (SNPs).
    Distributions given in handwritten notes.
    Note that pi_1 and pi_2 are always 0 (see handwritten notes, architectures).
    Returns (p x 1) vector of effect sizes u.
    """
    pi = np.random.rand(p)
    u = np.zeros(p)

    # vector of architectures for pi_0 and pi_3
    arch_vec = np.array([0.99, 0.95, 0.9, 0.8, 0.6, 0.4, 0.2, 0.1, 0.05, 0.01])
    
    for i in range(p):
        if pi[i] < arch_vec[arch]:
            # pi_3:
            u[i] = np.random.normal(0, np.sqrt(0.33 * 10**(-2)))
    
    return u

In [52]:
# Get the number of columns (p)
p = M_tilde.shape[1]

# Change based on which architecture is wanted
arch = 10

# Generate u values
u_sim = get_u(p = p, arch = arch-1)

Generate breeding values $a$.

In [48]:
# Compute a as matrix multiplication
a = np.dot(M_tilde.values, u_sim)

# Scale a to achieve heritability of 0.33
a = a * np.sqrt(0.33 / np.var(a))

Generate phenotype $y$.

In [49]:
# Generate epsilon (error term) with variance 0.67
epsilon = np.random.normal(0, np.sqrt(0.67), size=a.shape)

# Simulated phenotype y
y = a + epsilon

# Heritability check (approximately 0.33)
heritability = np.var(a) / np.var(y)
print(f"Heritability: {heritability:.2f}")

Heritability: 0.33


Plotting to inspect.

In [45]:
#plt.plot(y)

Storing phenotype and effect sizes locally.

In [ ]:
data_path = "C:/Users/gard_/Documents/MasterThesis/Code/Data/Phenotypes/70k/"
# Create a DataFrame for the phenotype and save it
y_df = pd.DataFrame({'Value': y})
y_df['ringnr'] = SNP_matrix['ringnr']
y_df.rename(columns={'Value': 'pheno'}, inplace=True)

# Save SNP effects
u_df = pd.DataFrame({'Value': u_sim})
u_df['SNP'] = M_tilde.columns
u_df.rename(columns={'Value': 'effect'}, inplace=True)


# Save to a CSV file
y_df.to_csv(data_path + "Sim_pheno_70k_arch_" + str(arch) + ".csv", index=False)

u_df.to_csv(data_path + "Sim_effect_70k_arch_" + str(arch) + ".csv", index=False)


# Display first few rows
print(y_df.head())
print(u_df.head())

      pheno   ringnr
0  0.026008  8L19501
1  0.435999  8L19502
2  1.610507  8L19503
3 -1.065995  8L19504
4  1.214641  8L19505
   effect           SNP
0     0.0  SNPa276757_T
1     0.0  SNPa283741_G
2     0.0  SNPa462043_A
3     0.0  SNPa462038_C
4     0.0  SNPa462037_A


Generate phenotypes and effect sizes for all $10$ architecture types in a for-loop.

In [ ]:
# Get the number of columns (p)
p = M_tilde.shape[1]

for i in range(10):
    arch = i + 1
    # Generate u values
    u_sim = get_u(p = p, arch = arch-1)

    # Compute a as matrix multiplication
    a = np.dot(M_tilde.values, u_sim)

    # Scale a to achieve heritability of 0.33
    a = a * np.sqrt(0.33 / np.var(a))

    # Generate epsilon (error term) with variance 0.67
    epsilon = np.random.normal(0, np.sqrt(0.67), size=a.shape)

    # Simulated phenotype y
    y = a + epsilon

    # Heritability check (approximately 0.33)
    heritability = np.var(a) / np.var(y)
    print(f"Heritability: {heritability:.2f}")

    data_path = "C:/Users/gard_/Documents/MasterThesis/Code/Data/Phenotypes/70k/"
    # Create a DataFrame for the phenotype and save it
    y_df = pd.DataFrame({'Value': y})
    y_df['ringnr'] = SNP_matrix['ringnr']
    y_df.rename(columns={'Value': 'pheno'}, inplace=True)

    # Save SNP effects
    u_df = pd.DataFrame({'Value': u_sim})
    u_df['SNP'] = M_tilde.columns
    u_df.rename(columns={'Value': 'effect'}, inplace=True)


    # Save to a CSV file
    y_df.to_csv(data_path + "Sim_pheno_70k_arch_" + str(arch) + ".csv", index=False)

    u_df.to_csv(data_path + "Sim_effect_70k_arch_" + str(arch) + ".csv", index=False)


    # Display first few rows
    print(y_df.head())
    print(u_df.head())

Heritability: 0.34
      pheno   ringnr
0  1.195317  8L19501
1  0.088351  8L19502
2  0.446012  8L19503
3 -0.668625  8L19504
4  0.040180  8L19505
     effect           SNP
0 -0.010230  SNPa276757_T
1 -0.071578  SNPa283741_G
2 -0.066896  SNPa462043_A
3 -0.035539  SNPa462038_C
4 -0.001448  SNPa462037_A
Heritability: 0.32
      pheno   ringnr
0  0.141303  8L19501
1 -0.954620  8L19502
2 -0.214632  8L19503
3 -0.421097  8L19504
4 -1.073395  8L19505
     effect           SNP
0 -0.048126  SNPa276757_T
1 -0.026093  SNPa283741_G
2 -0.084545  SNPa462043_A
3  0.097587  SNPa462038_C
4 -0.032629  SNPa462037_A
Heritability: 0.34
      pheno   ringnr
0 -0.789915  8L19501
1 -0.315160  8L19502
2  0.900878  8L19503
3 -1.295542  8L19504
4 -1.235822  8L19505
     effect           SNP
0  0.071783  SNPa276757_T
1  0.000000  SNPa283741_G
2 -0.006632  SNPa462043_A
3  0.000000  SNPa462038_C
4  0.000000  SNPa462037_A
Heritability: 0.33
      pheno   ringnr
0 -1.808461  8L19501
1 -0.591902  8L19502
2 -0.735930  8L

: 